# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

In [2]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Анатолий\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анатолий\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def lemmatize(text):
    lem = nltk.word_tokenize(text)
    lemmtext = [lemmatizer.lemmatize(i, get_wordnet_pos(i)) for i in nltk.word_tokenize(text)]
    return " ".join(lemmtext)

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [7]:
def clear_text(text):
    txt = re.sub(r'[^a-zA-Z0-9]',' ', text)
    txt = str(txt).lower()
    return ' '.join(txt.split())

In [8]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [9]:
data = pd.read_csv('toxic_comments.csv')

In [10]:
data.shape

(159571, 2)

In [11]:
data.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Тип данных нормально

In [13]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [14]:
#data = data.sample(15000)

In [ ]:
data['lemm_text'] = data['text'].apply(lambda x: lemmatize(x))

In [ ]:
data['lemm_text'] = data['lemm_text'].apply(lambda x: clear_text(x))

In [ ]:
data = data.drop(['text'], axis = 1)

In [ ]:
features = data.drop(['toxic'], axis = 1)

In [ ]:
target = data['toxic']

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [ ]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

In [ ]:
features_train.shape

In [ ]:
features_valid.shape

In [ ]:
features_test.shape

In [ ]:
target_train.shape

In [ ]:
target_valid.shape

In [ ]:
target_test.shape

In [ ]:
features_train, target_train = upsample(features_train, target_train, 9)

Разбил / Проверил

In [ ]:
corpus_train = features_train['lemm_text'].values.astype('U')

In [ ]:
corpus_valid = features_valid['lemm_text'].values.astype('U')

In [ ]:
corpus_test = features_test['lemm_text'].values.astype('U')

Корпус в юникод

In [ ]:
stopwords = set(nltk_stopwords.words('english'))

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [ ]:
count_tf_idf.fit(corpus_train)

Обучил только на трэйн

In [ ]:
tf_idf_train  = count_tf_idf.transform(corpus_train)

In [ ]:
tf_idf_valid = count_tf_idf.transform(corpus_valid)

In [ ]:
tf_idf_test = count_tf_idf.transform(corpus_test)

Преобразовал.

# Обучение / Модели

#### LogisticRegression

In [ ]:
model_LR = LogisticRegression(solver = 'liblinear', random_state = 12345)

In [ ]:
model_LR.fit(tf_idf_train, target_train)

In [ ]:
predict_LR = model_LR.predict(tf_idf_valid)

In [ ]:
f1_LR = f1_score(target_valid, predict_LR)

In [ ]:
f1_LR

#### RandomForestClassifier

In [ ]:
model_RFC =  RandomForestClassifier(n_estimators = 220, n_jobs = 4, random_state = 12345)

In [ ]:
model_RFC.fit(tf_idf_train, target_train)

In [ ]:
predict_RFC = model_RFC.predict(tf_idf_valid)

In [ ]:
f1_RFC = f1_score(target_valid, predict_RFC)

In [ ]:
f1_RFC

#### DecisionTreeClassifier

In [ ]:
model_DTC = DecisionTreeClassifier(max_depth = 96, random_state = 12345)

In [ ]:
model_DTC.fit(tf_idf_train, target_train)

In [ ]:
predict_DTC = model_DTC.predict(tf_idf_valid)

In [ ]:
f1_DTC = f1_score(target_valid, predict_DTC)

In [ ]:
f1_DTC

#### CatBoostClassifier

In [ ]:
model_CBC = CatBoostClassifier(iterations = 91, depth = 6, random_state = 12345)

In [ ]:
model_CBC.fit(tf_idf_train, target_train)

In [ ]:
predict_CBC = model_CBC.predict(tf_idf_valid)

In [ ]:
f1_CBC = f1_score(target_valid, predict_CBC)

In [ ]:
f1_CBC

#### XGBClassifier

In [ ]:
model_XGBC = xgb.XGBClassifier(max_depth = 71, n_estimators = 33, random_state = 12345)

In [ ]:
model_XGBC.fit(tf_idf_train, target_train)

In [ ]:
predict_XGBC = model_XGBC.predict(tf_idf_valid)

In [ ]:
f1_XGBC = f1_score(target_valid, predict_XGBC)

In [ ]:
f1_XGBC

# Выводы / Тест

Предсказания на тестовой выборке

In [ ]:
predict_LR_test = model_LR.predict(tf_idf_test)

In [ ]:
f1_LR_test = f1_score(target_test, predict_LR_test)

In [ ]:
f1_LR_test

In [ ]:
predict_RFC_test = model_RFC.predict(tf_idf_test)

In [ ]:
f1_RFC_test = f1_score(target_test, predict_RFC_test)

In [ ]:
f1_RFC_test

In [ ]:
predict_DTC_test = model_DTC.predict(tf_idf_test)

In [ ]:
f1_DTC_test = f1_score(target_test, predict_DTC_test)

In [ ]:
f1_DTC_test

In [ ]:
predict_CBC_test = model_CBC.predict(tf_idf_test)

In [ ]:
f1_CBC_test = f1_score(target_test, predict_CBC_test)

In [ ]:
f1_CBC_test

In [ ]:
predict_XGBC_test = model_XGBC.predict(tf_idf_test)

In [ ]:
f1_XGBC_test = f1_score(target_test, predict_XGBC_test)

In [ ]:
f1_XGBC_test

In [ ]:
table = pd.DataFrame(
    {'name': ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier', 'CatBoostClassifier', 'XGBClassifier']
                      , 'f1_score': [f1_LR, f1_RFC, f1_DTC, f1_CBC, f1_XGBC]
                      , 'f1_score_test': [f1_LR_test, f1_RFC_test, f1_DTC_test, f1_CBC_test, f1_XGBC_test]})

In [ ]:
table